In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
df = pd.read_pickle('data/data_after_feature_selection.pkl')

In [29]:
df = df[(sel.get_support())]
df.head()

ValueError: Item wrong length 19 instead of 199.

In [4]:
X = df.drop('Sztuk_sprzedanych_pierwsze_30_dni', axis=1)
y = df['Sztuk_sprzedanych_pierwsze_30_dni']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

print('Rozmiar zbioru: \nTreningowego: {}\nWalidacyjnego {}'.format(len(X_train), len(X_val)))

Rozmiar zbioru: 
Treningowego: 159
Walidacyjnego 40


# Decision Tree

In [5]:
y_val.std()

122.4659031563375

In [6]:
dt_reg = DecisionTreeRegressor(random_state=0, max_depth=2)
dt_reg.fit(X_train, y_train)
y_pred = dt_reg.predict(X_val)
sqrt(mean_squared_error(y_val, y_pred))

121.90577949638794

In [7]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_val)
sqrt(mean_squared_error(y_val, y_pred))

116.4829363491754

In [18]:
rf_reg = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=esti)
rf_reg.fit(X_train, y_train)
y_pred = rf_reg.predict(X_val)
print(sqrt(mean_squared_error(y_val, y_pred)))

49 99
124.83978281308848


In [22]:
sel = SelectFromModel(RandomForestRegressor(n_estimators = 50))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
        norm_order=1, prefit=False, threshold=None)

In [23]:
sel.get_support()

array([False, False, False,  True,  True, False, False, False, False,
       False, False, False, False,  True,  True, False,  True,  True,
       False])

In [24]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

6

In [25]:
selected_feat

Index(['top_seller_sales_30m', 'unique_sellers30m', 'top3_sales_30m',
       'ILE_AUKCJI_W_DNIU_DEBIUTU_FIRMA', 'sez_IND', 'time_untill_christmas'],
      dtype='object')

AttributeError: module 'pandas' has no attribute 'series'